## The Challenge

Consider a fund manager who interacts with the market through a number of custodians. Custodians hold client assets, but purchase/sale of those assets are under the direction of the fund manager. Fund manager makes orders to align client portfolios with particular strategies or intents, and places orders with relevant custodians.

On a daily basis the FM has a backlog of orders generated from some internal strategy. The FM goes through a placement process that results in placement of day orders with a number of custodians. 

At the end of day, the FM receives allocation feed files showing to which extent the day orders were filled and allocated to clients -- although in some cases the client allocation information is missing or partial, and in some cases the allocation response is absent/by voice. Allocations get converted to transactions (and therefore to holdings) for each client. 

The shortfall between allocation and outstanding order then forms the basis for the next day's order backlog.

## The Solution

Reconciliation between custodian allocations and outstanding orders depends on the allocations returned by custodians as being tagged with identifiers defined by the FM. Ideally the allocations are tagged with an FM order id, but could be tagged with something coarser, like client account name, or something finer, like day order id. In these cases the FM will need to update its orders with the necessary tags to enable rec (the full LUSID orders domain model will make placements available to record this information if needed).

This notebook is a reference implementation, a skeleton that shows how we support this kind of daily lifecycle *if* allocations are provided with all the infromation needed to fully reconcile against orders.

In this tutorial you will walk through how to set this up from a completely blank LUSID environment by:

1) Setting up a Scope to hold your portfolios

2) Creating your Instrument Universe 

3) Creating client portfolios

4) Setting your initial client holdings

5) Recording your initial outstanding order backlog

The daily cycle then is:

6) Recording the placement of orders with a number of custodians

7) Receive allocations from custodians at end of day

8) Convert allocations to transactions against client portfolios

9) Reconcile allocations against order backlog

10) Update orders to represent any shortfall for next day's backlog
*First things first run the cell below to import the libraries and authenticate your LUSID client*

In [ ]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
#from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
from collections import defaultdict
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json
import os

# Authenticate our user and create our API client
secrets_path = "/tmp/working/examples/secrets.json"
                
api_factory = lusid.utilities.ApiClientFactory(api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

## 1) Setting up a Scope to hold your portfolios

You will need to create a name for the scope that you will use for the fund. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [ ]:
# Give the scope a unique identifier
scope_id = import_data.create_scope_id()
# Give the scope a descriptive name
scope_name = 'UK_Equities_Fund'
# Join the two together to get the full scope name
scope = '{}_{}'.format(scope_name, scope_id)
prettyprint.heading('Scope', scope)

## 2) Creating your Instrument Universe 

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/instruments).

*Run the cell below to import your instrument universe*

In [ ]:
# Import your instrument universe from a CSV file
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument)

As part of this definition you will attach identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Schemes Should I Use With LUSID?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid)

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [ ]:
# Initialise a dictionary to hold your instrument definitions
instrument_definitions = {}

# Set the mapping between your identifier columns in the CSV and the available LUSID identifiers
identifier_columns = {
    'isin': 'Isin',
    'figi': 'Figi',
    'ticker': 'Ticker',
    'client_internal': 'ClientInternal'
}

# Iterate over your instrument universe
for index, instrument in instrument_universe.iterrows():

    # Initialise your set of identifiers for this instrument
    identifiers = {}
    
    # Populate your set of identifiers
    for identifier_column, identifier_lusid in identifier_columns.items():
        identifiers[identifier_lusid] = models.InstrumentIdValue(
            value=instrument[identifier_column])
        
    # Create the definition for your instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers
    )

# Call LUSID to upsert your instrument defintions as a batch
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    instruments=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

## 3) Creating client portfolios

To create a portfolio you need to give it a name and code - we'll create one for each client.

*Run the cell below to give your client portfolios a name and code*

In [ ]:
# Define portfolio ids
portfolios = {'Alice': 'Alice-portfolio-' + str(uuid.uuid4()), 'Bob': 'Bob-portfolio-' + str(uuid.uuid4())}

for client in portfolios:
    print(client + " " + portfolios[client])

Now that you have decided on the name and unique code for your portfolio you can create them. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/portfolios).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 365 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [ ]:
# Arbitrary portfolio creation date for illustration
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

for client in portfolios:
    # Create the request to add your portfolio to LUSID
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=client,
        code=portfolios[client],
        base_currency='USD',
        description='The portfolio for client ' + client,
        created=portfolio_creation_date)

    # Call LUSID to create your portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        transaction_portfolio=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(response)

## 4) Setting initial client holdings

Now that you have your instrument universe populated and client portfolios you can load your initial holdings into each. In this case you will import your holdings from a CSV file.

*Run the cell below to import your take on balances*

In [ ]:
#Import your holdings
holdings = pd.read_csv('data/fundmanager-initialholdings.csv')
holdings.head(10)

You can add these holdings to LUSID by setting the holdings on your portfolios. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The Effect of Holding Adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio). These holdings will be effective as of 4 days ago. This gives you some room to add recent transactions and conduct valuations.

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set the holdings on your client portfolios*

In [ ]:
# Set the effective date of these holdings to be 4 days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

holding_adjustments = defaultdict(list)

# Iterate over your holdings
for index, holding in holdings.iterrows():
    
    client = holding['client']

    # Create a holding adjustment for this holding
    holding_adjustments[client].append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                'Instrument/default/Figi': holding['figi']},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=holding['quantity'],
                    cost=models.CurrencyAndAmount(
                        amount=holding['total_cost'],
                        currency=holding['currency']),
                    portfolio_cost=holding['total_cost'],
                    price=holding['unit_cost'])
            ]
        )
    )
    
for holdings_client in holding_adjustments:
    
    # Call LUSID to set your holdings 
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).adjust_holdings(
        scope=scope,
        code=portfolios[holdings_client],
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments[holdings_client])

    # Pretty print the response 
    prettyprint.set_holdings_response(response, scope, portfolios[holdings_client])

## 5) Recording the order backlog

In [ ]:
#Define outstanding orders
orders = pd.read_csv('data/fundmanager-orders.csv')
orders.head(10)

In [ ]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

#Upsert orders
for index, order in orders.iterrows():
    
    client = order['client']

    order_requests[client].append(
        models.OrderRequest(
            code=order['order_id'],
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']},
            properties={},
            order_book=models.ResourceId(
                scope=scope,
                code='orderbook',
                ),
            portfolio=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )

for order_client in order_requests:
    order_sets[order_client] = models.OrderSetRequest(
        order_requests=order_requests[order_client]
        )

for order_client in order_sets:
    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        scope=scope,
        request=order_sets[order_client]
        )
    
    prettyprint.upsert_orders_response(response, scope)

# The daily cycle

## 6) Recording placement of orders with custodians

Once the backlog of orders has been established, the FM goes through a placement process in which orders are blocked together and placed with specific custodians.

Alice has funds with GS, while Bob has funds with MS.

Placements in this case would see ORD[001 - 004] placed with GS, and ORD[005 - 008] placed with MS. We can tag our orders to reflect the placement.

(Note that the full LUSID domain model will have Placement entities -- which will mean clients can form a more sophisticated/fine-grained view of their early trade lifecycle data).

In [ ]:
create_request = lusid.models.CreateDataTypeRequest(
    scope=scope,
    code="CustodianCodes",
    type_value_range="Closed",
    display_name="Available Custodian Tags",
    description="This data type contains the available Custodians which can be used",
    value_type="String",
    acceptable_values=[
        "GS",
        "MS"
    ])

response = api_factory.build(lusid.api.DataTypesApi).create_data_type(
    request=create_request)

print("Data Type Created")
prettyprint.heading("Scope", response.id.scope)
prettyprint.heading("Code", response.id.code)

In [ ]:
property_request = models.CreatePropertyDefinitionRequest(
    domain='Order',
    scope=scope,
    code='Custodian',
    value_required=False,
    display_name='Custodian',
    data_type_id=models.ResourceId(scope=scope, code='CustodianCodes'))

# Call LUSID to create our new property
property_response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
custodian_property_key = property_response.key

# Pretty print our key
prettyprint.heading('Custodian Property Key', custodian_property_key)

In [ ]:
custodians = { 'Alice': 'GS', 'Bob': 'MS' }

for index, order in orders.iterrows():
    
    client = order['client']
    order_requests = []
    
    order_requests.append(
        models.OrderRequest(
            code=order['order_id'],
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']},
            properties={
                     custodian_property_key: models.PerpetualProperty(
                         key=custodian_property_key,
                         value=models.PropertyValue(label_value=custodians[client])
                     )
                 },
            order_book=models.ResourceId(
                scope=scope,
                code='orderbook',
                ),
            portfolio=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
    order_set = models.OrderSetRequest(
        order_requests=order_requests
        )

    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        scope=scope,
        request=order_set
        )
    
    prettyprint.upsert_orders_response(response, scope)

## 7) Receive allocations from custodians at end of day

In [ ]:
#Read allocations
allocations = pd.read_csv('data/fundmanager-allocations.csv')
allocations.head(10)

In [ ]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

#Upsert allocations
for index, allocation in allocations.iterrows():
    
    client = allocation['client']

    allocation_requests[client].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=scope,
                code=allocation['allocation_id']
                ),
            allocated_order_id=models.ResourceId(
                scope=scope,
                code=allocation['originating_order']
                ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi']},
            properties={},
            portfolio_id=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
for allocation_client in allocation_requests:    
    allocation_sets[allocation_client] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_client]
        )

for allocation_client in allocation_sets:
    #Call LUSID to record your orders
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        request=allocation_sets[allocation_client]
        )
    
    prettyprint.upsert_allocations_response(response, scope)

## 8) Convert allocations to transactions against client portfolios

## 9) Reconcile allocations against order backlog

In [ ]:
# fetch orders
orders_response = api_factory.build(lusid.api.OrdersApi).list_orders(scope)

order_responses = {}
allocation_responses = {}

for order in orders_response.values:
    order_responses[order.id.code] = order

# fetch allocations
allocations_response = api_factory.build(lusid.api.AllocationsApi).list_allocations()

for allocation in allocations_response.values:
    allocation_responses[allocation.allocated_order_id.code] = allocation

print('OrderID\tIntrumentID\tQty\tAllocationID\tQty\tState')
for order in order_responses:
    shortfall = allocation_responses[order].quantity - order_responses[order].quantity
    state = "filled" if shortfall == 0 else "part. fill (" + str(shortfall) + ")"
    print (order + '\t' 
           + order_responses[order].lusid_instrument_id + '\t' 
           + str(order_responses[order].quantity) + '\t' 
           + allocation_responses[order].id.code + '\t' 
           + str(allocation_responses[order].quantity) + '\t' 
           + state)

## 10) Update orders to represent any shortfall for next day's backlog

In [ ]:
order_requests = []
    
for index, order in orders.iterrows():
    
    client = order['client']

    remaining = order['quantity'] - allocation_responses[order['order_id']].quantity
    
    order_requests.append(
        models.OrderRequest(
            code=order['order_id'],
            quantity=remaining,
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']},
            properties={
                     custodian_property_key: models.PerpetualProperty(
                         key=custodian_property_key,
                         value=models.PropertyValue(label_value=custodians[client])
                     )
                 },
            order_book=models.ResourceId(
                scope=scope,
                code='orderbook',
                ),
            portfolio=models.ResourceId(
                scope=scope,
                code=portfolios[client]
                )
            )
        )
    
order_set = models.OrderSetRequest(
    order_requests=order_requests
    )

#Call LUSID to record your orders
response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
    scope=scope,
    request=order_set
    )

prettyprint.upsert_orders_response(response, scope)